J&M Corporate Catalog and Business Dashboard
# **Data Upload:**
### **J&M CSVs to MySQL Database**

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### • **Import and Install the Necessary Libraries:**

In [ ]:
# Run if needed.

#!pip install pymysql
#!pip install sqlalchemy

In [ ]:
# Importing all the necessary libraries and modules.

import os
import sqlalchemy
SQLALCHEMY_SILENCE_UBER_WARNING = 1
import pandas as pd
import pymysql
import numpy as np
from sqlalchemy import create_engine, engine, text

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### • **Manage the Access to MySQL Database:**

#### - **Connect to the Database:**

In [ ]:
# Connecting to the database.

def connect_tcp_socket() -> sqlalchemy.engine.base.Engine:
    """ Initializes a TCP connection pool for a Cloud SQL instance of MySQL. """
    # For security reasons, these credentials were only included in Google App Engine.
    # ↓
    db_host = "" 
    db_user = "" 
    db_pass = "" 
    db_name = "" 
    db_port =  x

    engine = sqlalchemy.create_engine(
        sqlalchemy.engine.url.URL.create(
            drivername="mysql+pymysql",
            username=db_user,
            password=db_pass,
            host=db_host,
            port=db_port,
            database=db_name,
        ),
    )
    
    return engine

# Check: The engine was properly set up.

In [ ]:
# Calling the function and connecting to the engine.

engine = connect_tcp_socket() # engine will be MySQL
conn = engine.connect() #connect to MySQL server

#Check: The connection is settled.

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### • **Load the Data to MySQL Database:**

#### - **Read CSVs:**

In [ ]:
# Reading Customers Dataset as CSV.
customers_df = pd.read_csv('customers.csv')

#Check

In [ ]:
# Reading Customers Dataset as CSV
transactions_full_df = pd.read_csv('transactions.csv')

#Check

In [ ]:
# Reading Customers Dataset as CSV
articles_df = pd.read_csv('articles.csv')

#Check

#### - **Join Dataframe Columns to Create a Final Dataframe:**

In [ ]:
# Joining transactions_full_df with relevant article_df columns based on the article_id column
joined_df = transactions_full_df.merge(articles_df[['article_id', 'product_type_name', 'colour_group_name']], on='article_id', how='left')

# Joining the resulting dataframe with relevant customers_df columns based on the customer_id column
final_df = joined_df.merge(customers_df[['customer_id','club_member_status', 'fashion_news_frequency', 'age']], on='customer_id', how='left')

#### - **Upload the Data from the Dataframes to the MySQL Database:**

In [ ]:
# Uploading the Final Data Dataframe to the MySQL Database.

# Chunk size for uploading the dataframe to MySQL.
chunk_size = 100000

# Looping through the dataframe in chunks and uploading them to MySQL.
for i, chunk in enumerate(np.array_split(final_df, len(final_df) // chunk_size + 1)):
    chunk.to_sql(name='final_data', con=conn, if_exists='append', index=False)
    print(f'Uploaded chunk {i+1} of {len(final_df) // chunk_size + 1}')

#Check

#### - **Check the Existing Data in the MySQL Database:**

In [ ]:
# Checking if all the desired data was uploaded to the database.

result = conn.execute("SHOW TABLES;").fetchall()
for r in result:
    print(r)

#Check

#### - **Create a Users Table for Catalog Registration and Login Validation:**

In [ ]:
# Creating an empty table for storing and validating users' credentials.

query = conn.execute(""" DROP TABLE IF NOT EXISTS users (
        user_id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        username VARCHAR(255) NOT NULL,
        password VARCHAR(255) NOT NULL
        );
    """)

# Check: The table was successfully created with the desired field settings.

#### - **Confirm that the Users Credentials were Correctly Stored:**

In [ ]:
# Confirming that the user_id is incrementing by 1 and that the password is being stored in an hashing format.
users_data_df = load_data("""
SELECT * 
FROM users
""")

users_data_df

# Check: The user_id is incrementing by one with each user that registers and the password is being stored in an hashing format.